In [1]:
import time
import pandas as pd
from datetime import datetime
import threading

import bernielib as bl

bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [2]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [3]:
ber.robotHome()
ber.pipetteHome()

# Refill tips

In [4]:
ber.tips_rack.refill()

# Recalibrate pipette tip rack

In [46]:
ber.calibrateRack(rack='tips')

(77.2, 149.765, 168.9)

In [47]:
ber.move(z=50)

# Running a protocol

In [4]:
def waitAfterTimestamp(timestamp, delay):
    new_ts = time.time()
    while (new_ts - timestamp) < delay:
        time.sleep(1)
        new_ts = time.time()

In [5]:
def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        robot.dumpTipToWaste()
    
    return timestamp

In [6]:
def elutionMix(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    # Washing steps, moving along the wall
    # 1
    z_curr = z_top + 24
    #z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.0)
    robot.movePipetteToVolume(volume/4.0)
    time.sleep(delay/4.0)
    # 2
    z_curr = z_top + 28
    #z_curr = sample.calcAbsLiquidLevelFromVol(300, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    #robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(2 * (volume/4.0))
    time.sleep(delay/4.0)
    # 3
    z_curr = z_top + 32
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(3 * (volume/4.0))
    time.sleep(delay/4.0)
    # 4
    z_curr = z_top + 36
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(volume)
    time.sleep(delay/4.0)
    
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    z_curr = sample.calcAbsLiquidLevelFromVol(volume+100, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    robot.movePipetteToVolume(volume+50)
    time.sleep(delay)
    robot.movePipetteToVolume(0)

In [7]:
def elutionMixLrgVol(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    
    # Uptaking
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.8)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    
    # Ejecting liquid
    z_curr = sample.calcAbsLiquidLevelFromVol(1000, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.4)
    robot.movePipetteToVolume(volume+50)
    z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    # To back position
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    
    robot.movePipetteToVolume(0)

In [8]:
def separateEluate(robot, eluate_tube, result_tube, pipette_delay=0.5, z_above_bottom=0.5, safe_z=50):
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    # Uptaking liquid
    v = eluate_tube.getVolume()
    robot.moveToSample(eluate_tube)
    robot.movePipetteToVolume(v+5)
    robot.movePipetteToVolume(v)
    z0 = robot._getTubeZBottom(eluate_tube)
    robot.move(z=z0-z_above_bottom)
    robot.movePipetteToVolume(0)
    time.sleep(pipette_delay)
    # Dispensing liquid
    robot.moveToSample(result_tube)
    robot.movePipetteToVolume(v+5)
    time.sleep(pipette_delay)
    robot.movePipetteToVolume(200)
    time.sleep(pipette_delay)
    robot.touchWall(result_tube)
    robot.movePipetteToVolume(0)
    robot.move(z=safe_z)
    ber.dumpTipToWaste()

# Separating supernatant and eluate

In [22]:
def createSupernatantSamplesList(robot, number):
    samples_list = []
    for i in range(number):
        sample_name = 'sup'+str(i)
        s = bl.createSample('eppendorf', sample_name, robot.samples_rack, pos_col=1, pos_row=i+6, volume=0)
        samples_list.append(s)
    return samples_list

In [41]:
def removeSupAddEluent(robot, sample, sup, eluent, z_safe=50, delay=0.5, mix_times=6, T_pull=60):
    # Removing supernatant
    robot.moveMagnetsTowardsTube(poweroff=True)
    time.sleep(T_pull)
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 200:
        robot.movePipetteToVolume(200)
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        robot.move(z=z_safe)
        robot.dispenseLiquid(sup, 200, blow_extra=True)
        robot.move(z=z_safe)
    pipetteThread = threading.Thread(target=robot.movePipetteToVolume, args=(250,))
    pipetteThread.start()
    robot.moveToSample(sample)
    #robot.movePipetteToVolume(250)
    robot.move(z=z0)
    pipetteThread.join()
    robot.movePipetteToVolume(50)
    time.sleep(delay)
    robot.move(z=z0-0.5)
    time.sleep(delay)
    robot.moveAxisDelta('X', 1.5)
    robot.movePipetteToVolume(40)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', -1.5)
    robot.moveAxisDelta('Y', 1.5)
    robot.movePipetteToVolume(30)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('Y', -1.5)
    robot.moveAxisDelta('X', -1.5)    
    robot.movePipetteToVolume(20)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', 1.5)
    robot.moveAxisDelta('Y', -1.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay/4.0)
    robot.move(z=z_safe)
    robot.dispenseLiquid(sup, 250, blow_extra=True)
    robot.move(z=z_safe)
    robot.dumpTipToWaste()
    
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    robot.transferLiquid(eluent, sample, volume)
    for i in range(mix_times):
        elutionMixLrgVol(robot, sample, volume)
    elutionMix(robot, sample, volume)
    elutionMix(robot, sample, volume)
    elution_start_time = time.time()
    robot.move(z=z_safe)
    robot.dumpTipToWaste()
    return elution_start_time

In [42]:
sample_mix_scenario = pd.read_csv('mixing_pattern_eppendorf.csv')
beads_mix_scenario = pd.read_csv('mixing_pattern_eppendorf.csv')

In [44]:
initial_vol_list = [30, 30, 30, 30]
samples_number = len(initial_vol_list)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 900)

In [45]:
ber.pickUpNextTip()
samples_list = bl.createSamplesToPurifyList(ber, [0 for x in range(samples_number)])

for sample, volume in zip(samples_list, initial_vol_list):
    ber.transferLiquid(dna_stock_2, sample, volume, v_immerse_dispense=1600, touch_wall=False)

ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [46]:
V_avail_beads = 800 #uL
V_avail_water = 42000 #uL

T_absorb = 480 #sec
#T_absorb = 0 #sec
T_elution = 480 #sec
#T_elution = 0 #sec
T_pull = 60 #sec
#T_pull = 0 #sec

v_beads_list = [90, 27, 18, 15]
#v_beads_list = [90]
samples_list = bl.createSamplesToPurifyList(ber, initial_vol_list)
result_list = bl.createPurifiedSamplesList(ber, samples_number)
sup_list = createSupernatantSamplesList(ber, samples_number)
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 11, V_avail_beads)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, V_avail_water)

print("Experiment started ", datetime.now().strftime("%H:%M:%S"))
timestamp_beads_added = addBeadsToAll(ber, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario)
print("Waiting for DNA absorption")
waitAfterTimestamp(timestamp_beads_added, T_absorb)
print("DNA absorption finished ", datetime.now().strftime("%H:%M:%S"))
for sample, sup in zip(samples_list, sup_list):
    timestamp_elution_started = removeSupAddEluent(ber, sample, sup, water, T_pull=T_pull)
print("Eluent added ", datetime.now().strftime("%H:%M:%S"))
waitAfterTimestamp(timestamp_elution_started, T_elution)
print("Elution incubation finished ", datetime.now().strftime("%H:%M:%S"))
ber.moveMagnetsTowardsTube()
time.sleep(T_pull)
for sample, result in zip(samples_list, result_list):
        separateEluate(ber, sample, result)
print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))

Experiment started  19:46:20
Waiting for DNA absorption
DNA absorption finished  19:57:34
Eluent added  20:09:39
Elution incubation finished  20:17:30
Beads pulled to the side  20:20:36


In [29]:
ber.move(z=50)
ber.dumpTipToPosition(0, 5)

In [30]:
ber.move(z=50)


# Purging tubes

In [20]:
samples_list = bl.createSamplesToPurifyList(ber, [200,200,200,200])
waste = bl.createSample('50ml', 'liquid_waste', ber.reagents_rack, 0, 1, 0)
ber.pickUpNextTip()
ber.move(z=50)
for sample in samples_list:
    ber.transferLiquid(sample, waste, 200)
ber.move(z=50)    
ber.dumpTipToWaste()    

# Results analysis

No bands at any sample